In [2]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # 读取本地.env文件

# 初始化OLLAMA模型，指定模型名称和本地服务地址
model = OllamaLLM(model="deepseek-r1:1.5b", base_url="http://localhost:11434") 

In [3]:
def get_completion(prompt, model=model):
    # 创建一个提示模板
    prompt_template = PromptTemplate(
        input_variables=["input_text"],
        template="{input_text}"
    )
    # 创建一个LLMChain，将模型和提示模板关联起来
    chain = LLMChain(llm=model, prompt=prompt_template)
    # 运行LLMChain并返回结果
    return chain.run({"input_text": prompt})

# 使用OLLAMA模型生成文本
response = get_completion("what is 1+1?")
print(response)

C:\Users\33249\AppData\Local\Temp\ipykernel_38228\3423226809.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=model, prompt=prompt_template)
C:\Users\33249\AppData\Local\Temp\ipykernel_38228\3423226809.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return chain.run({"input_text": prompt})


<think>

</think>

1 + 1 equals **2**.


In [4]:
customer_email = """Arrr,I be fuming that me blender lid\
    flew off and splattered me kitchen walls\
    with smoothie! And to make matters worse,\
    the warranty don't cover the cost of\
    cleaning up me kitchen walls. I need yer help\
    right now, matey!
    """

style = """American English\
    in a calm and respectful tone\
    """

# 创建一个提示模板，将客户邮件和风格作为输入变量，并指定输出变量为翻译后的文本。
prompt = f"""Translate the text \
that is delimited by triple backticks\
into a style that is {style}.\
text: ```{customer_email}```
"""

# 使用OLLAMA模型生成翻译后的文本
response = get_completion(prompt)
print(response)

<think>
Okay, so the user wants me to translate a specific text into American English style. The text is enclosed in triple backticks and has some pretty strong words. Let's see... The original text is about someone blowing off their blender lid, splattering walls with smoothies, and complaining that the warranty doesn't cover cleanup costs.

First, I need to identify the key phrases: "blender lid," "splattered me kitchen walls," "warranty don't cover." These are critical because they convey frustration. The user wants this translated into a calm and respectful tone, so I'll need to soften the language while keeping the emotions intact.

"Blender lid" might be a bit harsh. Maybe something like "blender handle" or "blender cap" could work, but in an American context, using "blender handle" feels more natural. It's also less severe and maintains the essence of the original frustration without being too intense.

"splattered me kitchen walls" is pretty dark. To make it softer, I can say s

In [5]:
from langchain_core.prompts import ChatPromptTemplate,HumanMessagePromptTemplate

# 使用Ollama本地模型时，如果需要调节模型温度，需要在创建时就进行设置
# 如果要在运行中修改模型温度，则需要重新创建OllamaLLM实例
model = OllamaLLM(model="deepseek-r1:1.5b",base_url="http://localhost:11434",temperature=0.1)

# 使用消息模板列表初始化ChatPromptTemplate
# 因为ChatPromptTemplate必须要一个消息模板列表作为参数，
# 无法打印空的ChatPromptTemplate实例，所以这里打印一个空格字符串

# 定义一个 HumanMessagePromptTemplate
human_message_prompt = HumanMessagePromptTemplate.from_template("{input_text}")

# 使用消息模板列表初始化 ChatPromptTemplate
chat = ChatPromptTemplate.from_messages([human_message_prompt])

# 直接打印chat实例
print(chat) 


input_variables=['input_text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_text'], input_types={}, partial_variables={}, template='{input_text}'), additional_kwargs={})]


In [6]:
template_string = """Translate the text \
    that is delimited by triple backticks \
    into a style that is {style}.\
    text: ```{input_text}```"""

prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['input_text', 'style'], input_types={}, partial_variables={}, template='Translate the text     that is delimited by triple backticks     into a style that is {style}.    text: ```{input_text}```')

In [7]:
prompt_template.messages[0].prompt.input_variables

['input_text', 'style']

In [8]:
customer_style = """ American English\
    in a calm and respectful tone\
    """

customer_email = """Arrr,I be fuming that me blender lid\
    flew off and splattered me kitchen walls\
    with smoothie! And to make matters worse,\
    the warranty don't cover the cost of\
    cleaning up me kitchen walls. I need yer help\
    right now, matey!
    """
    
customer_messages = prompt_template.format_messages(
    style = customer_style,
    input_text = customer_email
)

print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages[0])

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
content="Translate the text     that is delimited by triple backticks     into a style that is  American English    in a calm and respectful tone    .    text: ```Arrr,I be fuming that me blender lid    flew off and splattered me kitchen walls    with smoothie! And to make matters worse,    the warranty don't cover the cost of    cleaning up me kitchen walls. I need yer help    right now, matey!\n    ```" additional_kwargs={} response_metadata={}


In [9]:
# ChatOpenAI 实现了 __call__ 方法，所以可以像函数一样被调用，用于和 OpenAI 的聊天模型交互。

# ChatPromptTemplate 主要用于构建和格式化提示消息，没有实现 __call__ 方法，不能直接调用，
# 需要先使用 format_messages 方法生成消息列表，再把消息列表传递给可调用的模型对象（如 ChatOpenAI 或 Ollama）来获取响应。

# invoke 方法是 langchain 里模型类提供的一个用于触发模型推理的方法。
# 当你调用 model.invoke(customer_messages) 时，实际上是在执行以下操作：
# 封装请求、发送请求、处理响应、返回结果。

#使用 model 与模型交互
customer_response = model.invoke(customer_messages)
print(customer_response)

<think>
Okay, so I have this query where someone wants me to translate a text delimited by triple backticks into American English with a calm and respectful tone. The text is pretty short and has some casual phrases.

First, I need to understand the context. The user provided an example response that translated the text into a more formal and polite manner. They used words like "arrr" which is slang for "arrrr," so maybe they meant to use that but it's not standard. Then, they changed "fuming" to "fogging" which seems like a typo or intentional change in tone.

I should consider the user's intent. They probably want the message to sound more professional and respectful, especially since it's addressed to someone else. The original text is a bit informal with some slang and casual language, so the translation needs to make that sound more polished.

Looking at the example response, they replaced "fuming" with "fogging," which makes sense as a typo correction but also changes the tone fr

In [10]:
service_reply = """Hey there customer,\
    the warranty does not cover \
    cleaning expenses for your kitchen \
    because it's your fault that \
    you misused your blender \
    by forgetting to put the lid on before \
    starting the blender. \
    Tough luck ! See ya !    
    """
    
service_style_pirate = """\
    a polite tone \
    that speaks in English Pirte \
    """
    
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    input_text=service_reply
)

print(service_messages[0].content)

service_response = model.invoke(service_messages)
print(service_response)

Translate the text     that is delimited by triple backticks     into a style that is     a polite tone     that speaks in English Pirte     .    text: ```Hey there customer,    the warranty does not cover     cleaning expenses for your kitchen     because it's your fault that     you misused your blender     by forgetting to put the lid on before     starting the blender.     Tough luck ! See ya !    
    ```
<think>
Okay, so I need to translate this text into a polite English style. Let me read through it again.

The user wrote: "Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck ! See ya !" 

Hmm, I notice some punctuation issues here. The closing quote is missing a period and a space after "See ya!" Maybe that should be "See ya" instead? Also, the word "tough luck" seems a bit informal. In polite English, it's better to use "g

In [11]:
{
    "gift":False,
    "delivery_days":5,
    "price_value":"pretty affordable !"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable !'}

In [12]:
customer_review = """\
    This leaf blower is pretty affordable \
    and does the job pretty well. \
    I would buy anthor one  for my friends Tom . \
    who needs a leaf blower. \
    It will take 2 days to arrive. \
    """

review_template = """\
    For the following text , extract the following information:
    things: what is the product?
    gift: is it a gift?
    delivery_days: how many days does it take to deliver?
    
    Format the output as JSON with the following keys:
    things
    gift
    delivery_days
    
    text: {text}
    """

In [ ]:
from langchain.prompts import ChatPromptTemplate,HumanMessagePromptTemplate

model = OllamaLLM(model="deepseek-r1:1.5b",base_url="http://localhost:11434",temperature=0.1)

# 调用format_messages时要为模板中的所有占位符（即{text},{extra_info}）传入对应的值，
# 这样才能保证messages具有完整的内容

prompt_template = ChatPromptTemplate.from_template(review_template)
messages = prompt_template.format_messages(text=customer_review)
print(messages[0].content)

response = model.invoke(messages)

print(response)

    For the following text , extract the following information:
    things: what is the product?
    gift: is it a gift?
    delivery_days: how many days does it take to deliver?
    
    Format the output as JSON with the following keys:
    things
    gift
    delivery_days
    
    text:     This leaf blower is pretty affordable     and does the job pretty well.     I would buy anthor one  for my friends Tom .     who needs a leaf blower.     It will take 2 days to arrive.     
    
<think>
Okay, so I need to extract information from this text about the product, whether it's a gift, and how many delivery days there are. Let me read through the text carefully.

The text starts with "This leaf blower is pretty affordable" – that seems like the price aspect. Then it says "and does the job pretty well." That probably means it's good for its price. Next, it mentions "I would buy anthor one  for my friends Tom . who needs a leaf blower." So, I think "anthor" is the brand name here.

The t

In [14]:
type(response)

str

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

things_schema = ResponseSchema(name="things",description="what is the product?")
gift_schema = ResponseSchema(name="gift",description="is it a gift?") #  定义一个名为 gift_schema 的 ResponseSchema 对象，用于描述是否为礼物
delivery_days_schema = ResponseSchema(name="delivery_days",description="how many days does it take to deliver?") #  定义一个名为 delivery_days_schema 的 ResponseSchema 对象，用于描述配送所需天数

response_schemas = [things_schema,gift_schema,delivery_days_schema] #  创建一个列表 response_schemas，包含之前定义的 things_schema、gift_schema 和 delivery_days_schema
print(response_schemas)

[ResponseSchema(name='things', description='what is the product?', type='string'), ResponseSchema(name='gift', description='is it a gift?', type='string'), ResponseSchema(name='delivery_days', description='how many days does it take to deliver?', type='string')]


In [16]:
# 实例化 StructuredOutputParser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"things": string  // what is the product?
	"gift": string  // is it a gift?
	"delivery_days": string  // how many days does it take to deliver?
}
```


In [ ]:
review_template_2 = """\
    For the following text , extract the following information:
    
    things: what is the product?
    gift: is it a gift?
    delivery_days: how many days does it take to deliver?

    Format the output as JSON with the following keys:
    things
    gift
    delivery_days

    text: {text}
    {format_instructions}
    """
prompt_template_2 = ChatPromptTemplate.from_template(template=review_template_2) #  使用ChatPromptTemplate类从上述模板创建一个聊天提示模板对象
messages = prompt_template_2.format_messages( #  使用format_messages方法格式化消息，传入客户评论和格式指令 customer_review是客户提供的评论文本 format_instructions是格式化输出的指令
            text=customer_review,format_instructions=format_instructions)

In [23]:
print(messages[0].content)

    For the following text , extract the following information:
    
    things: what is the product?
    gift: is it a gift?
    delivery_days: how many days does it take to deliver?

    Format the output as JSON with the following keys:
    things
    gift
    delivery_days

    text:     This leaf blower is pretty affordable     and does the job pretty well.     I would buy anthor one  for my friends Tom .     who needs a leaf blower.     It will take 2 days to arrive.     
    The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"things": string  // what is the product?
	"gift": string  // is it a gift?
	"delivery_days": string  // how many days does it take to deliver?
}
```
    


In [33]:
import re

try:
    # 调用模型生成响应
    response_text = model.predict(messages[0].content) #  使用模型的predict方法生成响应文本

    # 提取 JSON 部分
    json_pattern = r'```json(.*?)```' #  定义正则表达式模式，用于匹配以```json开头和结尾的部分
    match = re.search(json_pattern, response_text, re.DOTALL) #  在响应文本中搜索匹配的部分，re.DOTALL使.匹配包括换行符在内的所有字符

    if match:
        json_text = match.group(1).strip() #  提取匹配到的JSON文本，并去除首尾的空白字符
        output_dict = output_parser.parse(f"```json\n{json_text}\n```") #  使用output_parser解析JSON文本
        print(output_dict) #  打印解析后的字典
    else:
        print("未找到有效的 JSON 部分。") #  如果未找到匹配的JSON部分，打印提示信息
except Exception as e:
    print(f"发生错误: {e}") #  捕获并打印任何异常信息
    

{'things': 'leaf blower', 'gift': False, 'delivery_days': 2}


In [37]:
print(type(output_dict))
print(output_dict.get("gift"))

<class 'dict'>
False
